In [3]:
import pandas as pd

# 讀取 df = perfect資料json檔 和 df_oscar = oscar csv檔

df = pd.read_json('4_perfer_movie_7620.json')
df_oscar = pd.read_csv('6.1_the_oscar_award.csv')

# df_oscar 第一直欄 有多的index 丟掉 

df_oscar.drop(df_oscar.columns[0],axis = 1,inplace= True)

# 重置df資料索引

df.reset_index(drop = True , inplace = True)

# df的cast直欄內容是list型態轉成字串型態

df['casts']= df['casts'].astype('string')

# df生成6個actor直欄

for number in range(6):
    
# 用正則取代  ] [ ' 三個符號 再用 , 切割成6個直欄 ,再轉成小寫 
    df[f'actor{number + 1}'] =  df['casts'].str.replace("[\]\[\']",'',regex = True).str.split(',',expand=True).loc[:,number].str.lower()
    
# 用pd.datetime取出年分

df['year'] = pd.to_datetime(df['release_date']).dt.year

# 轉小寫

df['title_sum_reg_new'] = df['title_sum_reg_new'].str.lower()

# df_new 拿要比對的直欄就好

df_new = df[['title_sum_reg_new','year','actor1','actor2','actor3','actor4','actor5','actor6']]

# df_new actor{i}要把前後空格去掉
for i in range(1,7):
    df_new[f'actor{i}'] = df_new[f'actor{i}'].str.strip()

# df生成6個actor_count直欄

for number in range(1,7):
    df_new[f'actor{number}_count'] = 0
    
# df_oscar 只留男女演員獎項

df_oscar_new = df_oscar[df_oscar["category"].isin(['ACTOR IN A LEADING ROLE','ACTRESS IN A LEADING ROLE','ACTRESS IN A SUPPORTING ROLE','ACTOR IN A SUPPORTING ROLE'])]

# df_oscar 轉小寫

df_oscar_new['name'] = df_oscar_new['name'].str.lower()

# 重置索引

df_oscar_new.reset_index(drop = True , inplace = True)

# 精簡版亂

# list1 = []
# for index,obj in enumerate (df_new['actor1']) :
#     if (df_oscar_new['name'] == obj).sum() >= 1 :
#         value = df_oscar_new.loc[(df_oscar_new['name'] == obj)&(df_oscar_new['year_film']  <= df_new['year'][index]),'year_film'].count()
#         list1.append(value)
#     else:
#         list1.append(0)

# 易讀版



for i in range(1,7):
    list1 = []
    for index, actorname in enumerate(df_new[f'actor{i}']):
        
    # 如果df的演員名稱 有跟 oscar 的 片名一樣 進行下列程式
        if (df_oscar_new['name'] == actorname).sum() >= 1 :
            
    # 得到該電影的 上映年分
            movie_release_year = df_new['year'][index]
    #  條件式=>大表的演員名字 和 奧斯卡演員命字一樣 ..... 奧斯卡得獎年分 小於等於 電影上映日期 
            filter1 = (df_oscar_new['name'] == actorname)&(df_oscar_new['year_film']  <= movie_release_year)&(df_oscar_new['winner'] == 1)
    # 根據上述條件式會得到一個series series 內容等於該電影第一演員 在這部電影上映時 以前得獎清單的各年分 
    # 再用count()計算這清單有多少筆數據就是這演員的在該年以前的得獎數字
            value = df_oscar_new.loc[filter1,'year_film'].count()
            list1.append(value)
        else:
            list1.append(0)
    
    df_new[f'actor{i}_count'] = list1

C:\Users\ASUS\AppData\Local\Temp/ipykernel_7624/93723547.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[f'actor{i}'] = df_new[f'actor{i}'].str.strip()
C:\Users\ASUS\AppData\Local\Temp/ipykernel_7624/93723547.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[f'actor{number}_count'] = 0
C:\Users\ASUS\AppData\Local\Temp/ipykernel_7624/93723547.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [4]:
for i in range(1,7):
    print(df_new[f'actor{i}_count'].value_counts())

0    6871
1     578
2     165
3       6
Name: actor1_count, dtype: int64
0    7040
1     502
2      77
3       1
Name: actor2_count, dtype: int64
0    7221
1     357
2      40
3       2
Name: actor3_count, dtype: int64
0    7327
1     261
2      30
3       2
Name: actor4_count, dtype: int64
0    7433
1     164
2      22
3       1
Name: actor5_count, dtype: int64
0    7482
1     129
2       8
3       1
Name: actor6_count, dtype: int64


In [6]:
df_new.to_json('7620_count_json_0325.json')